<a href="https://colab.research.google.com/github/nandini11122004/Analog_automation/blob/main/BISTM_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, Layer, Flatten
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import GlobalAveragePooling1D
from sklearn.preprocessing import LabelEncoder

# Load component dictionary from file
import pandas as pd

# Circuit categories and their expected component context
circuit_data = {
    "amplifier": [
        ["Vdd", "Vin", "R", "Q", "Cshunt"],
        ["Voltage Supply", "Voltage Input", "Resistor", "NPN Transistor", "Capacitor Shunt"]
    ],
    "oscillator": [
        ["Vdd", "L", "C", "Q", "R"],
        ["Voltage Supply", "Inductor", "Capacitor", "PNP Transistor", "Resistor"]
    ],
    "rectifier": [
        ["Vin", "D", "R", "C"],
        ["Voltage Input", "Diode", "Resistor", "Capacitor"]
    ],
    "regulator": [
        ["Vin", "Vout", "R", "C", "IC"],
        ["Voltage Input", "Voltage Output", "Resistor", "Capacitor", "Integrated Circuit"]
    ],
    "filter1": [
        ["Vin", "C", "L", "R"],
        ["Voltage Input", "Capacitor", "Inductor", "Resistor"]
    ],
    "filter2": [
        ["XU", "R", "C", "Vn", "Vp", "Vin"],
        ["Regulator Integrated Circuit", "Resistor", "Capacitor", "Negative Voltage", "Positive Voltage", "Voltage Input"]
    ],
    "current_mirror": [
        ["V", "I", "M", "M", "R", "nmos"],
        ["Voltage Source", "Current Source", "MOSFET", "MOSFET", "Resistor", "MOSFET"]
    ],
    "voltage_regulator": [
        ["XU", "R", "C", "R", "Vin", "Vin", "Vout"],
        ["Regulator Integrated Circuit", "Resistor", "Capacitor", "Resistor", "Voltage Input", "Voltage Input", "Voltage Output"]
    ],
    "buck_converter": [
        ["Vin", "L", "C", "D", "Q"],
        ["Voltage Input", "Inductor", "Capacitor", "Diode", "MOSFET"]
    ],
    "boost_converter": [
        ["Vin", "L", "C", "D", "Q"],
        ["Voltage Input", "Inductor", "Capacitor", "Diode", "MOSFET"]
    ],
    "buck_boost_converter": [
        ["Vin", "L", "C", "D", "Q"],
        ["Voltage Input", "Inductor", "Capacitor", "Diode", "MOSFET"]
    ],

    "pll": [
        ["Vco", "PFD", "Loop_Filter", "Divider"],
        ["Voltage Controlled Oscillator", "Phase Frequency Detector", "Loop Filter", "Frequency Divider"]
    ],
    "mixer": [
        ["RF", "LO", "IF", "Q"],
        ["Radio Frequency Input", "Local Oscillator", "Intermediate Frequency Output", "Transistor"]
    ],
    "modulator": [
        ["Vin", "Vout", "Q", "C", "R"],
        ["Voltage Input", "Voltage Output", "Transistor", "Capacitor", "Resistor"]
    ],
    "demodulator": [
        ["Vin", "D", "C", "R"],
        ["Voltage Input", "Diode", "Capacitor", "Resistor"]
    ],
    # Additional Circuits
    "digital_buffer": [
        ["Vin", "Vout", "IC"],
        ["Voltage Input", "Voltage Output", "Buffer IC"]
    ],

    "rf_amplifier": [
        ["Vin", "Vout", "Q", "L", "C"],
        ["RF Input", "RF Output", "Transistor", "Inductor", "Capacitor"]
    ],
    "log_amplifier": [
        ["Vin", "Vout", "OpAmp", "D"],
        ["Voltage Input", "Voltage Output", "Operational Amplifier", "Diode"]
    ],
    "precision_rectifier": [
        ["Vin", "Vout", "OpAmp", "D"],
        ["Voltage Input", "Voltage Output", "Operational Amplifier", "Diode"]
    ],
    "comparator": [
        ["Vin+", "Vin-", "Vout", "OpAmp"],
        ["Non-Inverting Input", "Inverting Input", "Voltage Output", "Operational Amplifier"]
    ],
    "frequency_synthesizer": [
        ["VCO", "PLL", "Mixer", "Divider"],
        ["Voltage Controlled Oscillator", "Phase-Locked Loop", "Mixer", "Frequency Divider"]
    ],
    "rf_mixer": [
        ["RF", "LO", "IF", "Q", "D"],
        ["Radio Frequency Input", "Local Oscillator", "Intermediate Frequency Output", "Transistor", "Diode"]
    ],
    "led_driver": [
        ["Vin", "LED", "R", "Q"],
        ["Voltage Input", "Light Emitting Diode", "Resistor", "Transistor"]
    ],
    "motor_driver": [
        ["Vin", "Motor", "H-Bridge", "PWM"],
        ["Voltage Input", "Motor", "H-Bridge Circuit", "Pulse Width Modulation"]
    ],
    "delta_sigma_modulator": [
        ["Vin", "Vout", "Integrator", "Comparator", "D Flip-Flop"],
        ["Voltage Input", "Voltage Output", "Integrator Circuit", "Comparator", "D-Type Flip-Flop"]
    ]
}


# Tokenization
#frequency based tokenization
# Step 1: Extract unique components
all_components = list(set(comp for variations in circuit_data.values() for components in variations for comp in components))

# Step 2: Shuffle components randomly
random.shuffle(all_components)

# Step 3: Assign random indices to each component (ensuring uniqueness)
random_token_map = {comp: idx + 1 for idx, comp in enumerate(all_components)}  # +1 to avoid index 0 (used for padding)

# Step 4: Find maximum sequence length
max_length = max(len(components) for variations in circuit_data.values() for components in variations)

# Step 5: Convert components to token sequences
X = [[random_token_map[comp] for comp in components] for variations in circuit_data.values() for components in variations]

# Step 6: Pad sequences
X = pad_sequences(X, maxlen=max_length, padding='post')

# Step 7: Define vocabulary size
vocab_size = len(random_token_map) + 1  # Include 0 for padding



# Encode labels
label_encoder = LabelEncoder()
y_labels = [label for label, variations in circuit_data.items() for _ in variations]
y = label_encoder.fit_transform(y_labels)
y = tf.keras.utils.to_categorical(y, num_classes=len(set(y_labels)))

# Define Attention Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,), initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
      e = tf.nn.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
      a = tf.nn.softmax(e / 0.5, axis=1)  # Divide by 0.5 to reduce overconfidence
      weighted_sum = tf.reduce_sum(x * a, axis=1)
      return weighted_sum, a

input_layer = Input(shape=(max_length,))
embedding_layer = Embedding(input_dim=vocab_size, output_dim=128, embeddings_regularizer=tf.keras.regularizers.l2(1e-5))(input_layer)

# Bi-LSTM Layer
bi_lstm = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)

# Get both the weighted sum and attention scores
weighted_sum, attention_scores = AttentionLayer(name="attention_layer")(bi_lstm)


# Classification layer
output_layer = Dense(len(set(y_labels)), activation="softmax")(weighted_sum)

# Define model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X, y, epochs=50, batch_size=4, verbose=1)


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.0000e+00 - loss: 3.2212
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1582 - loss: 3.2062
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2616 - loss: 3.1840
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5491 - loss: 3.1574
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5596 - loss: 3.1229
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5466 - loss: 3.0541
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3603 - loss: 2.9178
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4962 - loss: 2.5379
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4437 - loss: 2.1778
Epoch 10/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4927 - loss: 1.9141
Epoch 11/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.6721 - loss: 1.6128
Epoch 12/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accu

In [2]:
def classify_circuit(components):
    # Convert components to tokenized sequence using the randomized token map
    seq = [[random_token_map[comp] for comp in components if comp in random_token_map]]

    # Pad sequence
    seq_padded = pad_sequences(seq, maxlen=max_length, padding='post')

    # Predict probabilities
    probabilities = model.predict(seq_padded)[0]

    # Convert NumPy types to Python types for JSON compatibility
    circuit_probabilities = {str(label_encoder.inverse_transform([i])[0]): float(prob) for i, prob in enumerate(probabilities)}

    # Sort probabilities in descending order and get the top 5
    top_5 = sorted(circuit_probabilities.items(), key=lambda x: x[1], reverse=True)[:5]

    return top_5

# Example: Classify a new circuit
new_components = ["Vin", "D", "R"]
top_5_probabilities = classify_circuit(new_components)

# Print top 5 predictions
for circuit, prob in top_5_probabilities:
    print(f"{circuit}: {prob:.4f}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
rectifier: 0.9403
demodulator: 0.0247
led_driver: 0.0163
filter1: 0.0048
modulator: 0.0044
